In [9]:
import os

try:
    from common.sagemaker_env import LABELBOX_API_KEY
except: 
    LABELBOX_API_KEY = os.environ['LABELBOX_API_KEY']
    
    

CLOUD_PROJECT_ID = "cleamnf3q398707ug5s2z4rp6"

In [105]:

labelbox_dir = "./data/labelbox"
clouds_dir = f'{labelbox_dir}/clouds'
source_dir = f'{clouds_dir}/source'
training_dir = f'{clouds_dir}/training'
labels_dir = f'{training_dir}/labels'
rgb_dir = f'{clouds_dir}/rgb'


In [3]:

def is_island(target_polygon, polygon_list):
    for curr_poly in polygon_list:
        if target_polygon.within(curr_poly):
            return True
        
    return False


## OLD - use parallel version

In [ ]:
%reload_ext autoreload
%autoreload 2

import glob
import numpy as np
from patchify import patchify
import rasterio


from common.constants import NODATA_BYTE, NODATA_FLOAT32
import common.utilities.imagery as imagery
import common.utilities.visualization as visualization


import matplotlib.pyplot as plt


PATCH_SIZE = 512

scene_dirs = glob.glob(f'./data/20*/S2*')

for i, scene_dir in enumerate(scene_dirs):
    
    scene = scene_dir.split('/')[-1]
    print(scene)
    
    with rasterio.open(f'{scene_dir}/B08.tif') as src:
        if src.width < 512 or src.height < 512:
            continue
    
    stack_data = []
    transform = None
    tif_paths = sorted(glob.glob(f'{scene_dir}/B*.tif'))
    for path in tif_paths:
        with rasterio.open(path) as src:
            transform = src.transform
            stack_data.append(src.read(1))
            
    stack_data = np.ma.array(stack_data)
    stack_data = imagery.normalize_3d_array(stack_data).transpose((1, 2, 0))
        
    source_patches = patchify(stack_data, (PATCH_SIZE, PATCH_SIZE, stack_data.shape[2]), step=PATCH_SIZE)
    
    for irow in range(source_patches.shape[0]):
        for icol in range(source_patches.shape[1]):
            source_data = source_patches[irow, icol, 0, :, :, :]            
            source_data = np.ma.array(source_data, mask=(source_data==NODATA_FLOAT32))
            
            if source_data.mask.sum() > 0:
                print('skipping:', irow, icol)
                continue
                
            #if irow < 3 or icol < 3:
            #    continue

            rgb_data = source_data[:, :, [2, 1, 0]]
            rgb_data_norm = np.round(np.multiply(rgb_data, 254)).astype(int)
            rgb_data_norm2 = imagery.normalize_0_254_3d_array(rgb_data)
            
            #print('\t', np.min(rgb_data_norm), np.median(rgb_data_norm), np.max(rgb_data_norm))
            #print('\t', np.min(rgb_data_norm2), np.median(rgb_data_norm2), np.max(rgb_data_norm2))
            #print('\t-------------')
            
            #fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 12))
            #visualization.plot_bands(rgb_data1, ax=ax1, bands=[0, 1, 2], transpose=False)
            #visualization.plot_bands(rgb_data2, ax=ax2, bands=[0, 1, 2], transpose=False)
            #raise
                
            source_path = f'{source_dir}/{scene}_{irow}_{icol}.tif'
            rgb_path = f'{rgb_dir}/{scene}_{irow}_{icol}.tif'
                    
            x_min, y_min = rasterio.transform.xy(transform, PATCH_SIZE*(irow+1), PATCH_SIZE*icol)
            x_max, y_max = rasterio.transform.xy(transform, PATCH_SIZE*irow, PATCH_SIZE*(icol+1))             
            bbox = [x_min, y_min, x_max, y_max]
            
            imagery.write_array_to_tif(rgb_data_norm, rgb_path, bbox, dtype=np.uint8, nodata=255, is_cog=True) 
            imagery.write_array_to_tif(source_data, source_path, bbox, dtype=np.float32, nodata=NODATA_FLOAT32) 
                   

S2A_35MQV_20200731_0_L2A
S2A_35MQV_20200820_0_L2A
S2A_35MQU_20211123_0_L2A
S2B_35MQU_20211108_0_L2A
S2B_35MQU_20211029_0_L2A


KeyboardInterrupt: 

## Create scene patches in parallel

In [36]:
%reload_ext autoreload
%autoreload 2

import glob
import multiprocessing
import numpy as np
from patchify import patchify
import rasterio


from common.constants import NODATA_BYTE, NODATA_FLOAT32
import common.utilities.imagery as imagery
import common.utilities.visualization as visualization



PATCH_SIZE = 512


def process_scene(scene_dir):
    
    scene = scene_dir.split('/')[-1]
    
    with rasterio.open(f'{scene_dir}/B08.tif') as src:
        if src.width < 512 or src.height < 512:
            return

    stack_data = []
    transform = None
    tif_paths = sorted(glob.glob(f'{scene_dir}/B*.tif'))
    for path in tif_paths:
        with rasterio.open(path) as src:
            transform = src.transform
            stack_data.append(src.read(1))

    stack_data = np.ma.array(stack_data)
    stack_data = imagery.normalize_3d_array(stack_data).transpose((1, 2, 0))
        
    source_patches = patchify(stack_data, (PATCH_SIZE, PATCH_SIZE, stack_data.shape[2]), step=PATCH_SIZE)
    
    for irow in range(source_patches.shape[0]):
        for icol in range(source_patches.shape[1]):
            source_data = source_patches[irow, icol, 0, :, :, :]            
            source_data = np.ma.array(source_data, mask=(source_data==NODATA_FLOAT32))
            
            if source_data.mask.sum() > 0:
                print('skipping:', irow, icol)
                continue
                
            rgb_data = source_data[:, :, [2, 1, 0]]
            rgb_data_norm = np.round(np.multiply(rgb_data, 255)).astype(np.uint8) + 1
            
            source_path = f'{source_dir}/{scene}_{irow}_{icol}.tif'
            rgb_path = f'{rgb_dir}/{scene}_{irow}_{icol}.tif'
                    
            x_min, y_min = rasterio.transform.xy(transform, PATCH_SIZE*(irow+1), PATCH_SIZE*icol)
            x_max, y_max = rasterio.transform.xy(transform, PATCH_SIZE*irow, PATCH_SIZE*(icol+1))             
            bbox = [x_min, y_min, x_max, y_max]
            
            imagery.write_array_to_tif(rgb_data_norm, rgb_path, bbox, dtype=np.uint8, nodata=0, is_cog=True) 
            imagery.write_array_to_tif(source_data, source_path, bbox, dtype=np.float32, nodata=NODATA_FLOAT32) 
                       
            
            
scene_dirs = glob.glob(f'./data/20*/S2*')
print(f'{len(scene_dirs)} scenes')
with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
    patches = pool.map(process_scene, scene_dirs)
    

103 scenes


In [29]:
%%bash

gdalinfo -stats ./data/labelbox/clouds/rgb/S2A_35MQV_20200731_0_L2A_0_0.tif


Driver: GTiff/GeoTIFF
Files: ./data/labelbox/clouds/rgb/S2A_35MQV_20200731_0_L2A_0_0.tif
Size is 512, 512
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (29.242784885946573,-0.591630233123690)
Pixel Size = (0.000089771893148,-0.000090466247379)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  INTERLEAVE=PIXEL
  LAYOUT=COG
Corner Coordinates:
Upper Left  (  29.2427849,  -0.5916302) ( 29d14'34.03"E,  0d35'29.87"S)
Lower Left  (  29.2427849,  -0.6379490) ( 29d14'3

## Upload to S3

In [37]:
%reload_ext autoreload
%autoreload 2

import common.constants as constants
import common.aws.s3 as s3_utils


def save_patch_to_s3(tif_path):
    
    file_name = tif_path.split('/')[-1]    
    object_key = f'clouds/{file_name}'
    href = f'https://data.smartcarte.earth/{object_key}'
    # print(f'uploading {tif_path} to s3://{constants.S3_DATA_BUCKET}/{object_key}')
    s3_utils.put_item(tif_path, constants.S3_DATA_BUCKET, object_key)
    
    return href


In [87]:
import glob
import shelve


state_path = "./data/trainCloudsState"
with shelve.open(state_path) as state:

    rgb_paths = glob.glob(f'{rgb_dir}/*.tif')
    rgb_paths = [path for path in rgb_paths if not path.endswith("deg.tif")]

    for i, path in enumerate(rgb_paths):
        
        if i % 200 == 0:
            print(f'{i} done')
            
        job_name = path.split('/')[-1].replace('.tif', '')
        s3_href = save_patch_to_s3(path)
        state[job_name] = {
            'rgb_cog_href': s3_href
        }
    

3633 paths
3632 paths


RuntimeError: No active exception to reraise

In [39]:
import datetime
import glob
import json
from labelbox import Client, Dataset, DataRow
import os
import random
import shelve
from uuid import uuid4


client = Client(api_key=LABELBOX_API_KEY)
project = client.get_project(CLOUD_PROJECT_ID)

today = datetime.datetime.today().strftime('%Y%m%d')

clouds_dataset_name = f"{today} Clouds"

clouds_dataset = client.get_datasets(where=(Dataset.name==clouds_dataset_name)).get_one()
if clouds_dataset is None:
    clouds_dataset = client.create_dataset(name=clouds_dataset_name)
else:
    raise ValueError("today's clouds dataset already exists")

    
payload = []
with shelve.open(state_path) as state:
    for job_name in state.keys():        
        row = {
            "min_zoom": 12,
            "max_zoom": 14,
            "tile_layer_url": state[job_name]["rgb_cog_href"]
        }
        
        payload.append(row)


payload = [{DataRow.row_data: row} for row in payload]

random.seed(666)
random.shuffle(payload)

start_idx, end_idx = 0, 1000
payload = payload[start_idx:end_idx]

print(f'{len(payload)} data rows')
task = clouds_dataset.create_data_rows(payload)
task.wait_till_done()

print('done')

1000 data rows
done


In [ ]:
files = glob.glob("./data/labelbox/clouds/source/*.tif")
print(len(files))

664


### Pull data from Labelbox

In [71]:
from datetime import datetime, timedelta
from labelbox import Client


def get_data_rows():
    client = Client(api_key=LABELBOX_API_KEY)
    project = client.get_project(CLOUD_PROJECT_ID)

    now = datetime.today()
    then = now - timedelta(days=60)

    data_rows = project.export_labels(download=True) #, start=then.strftime('%Y-%m-%d'), end=now.strftime('%Y-%m-%d'))    
    return data_rows


def is_island(target_polygon, polygon_list):
    for curr_poly in polygon_list:
        if target_polygon.within(curr_poly):
            return True
        
    return False


In [75]:
import json
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from rasterio.features import geometry_mask
from shapely.geometry import Polygon


from common.utilities.imagery import write_array_to_tif
from common.utilities.visualization import plot_bands


CLOUD_CLASSES = {
    'no_cloud': 0,
    'cloud': 1
}

data_rows = get_data_rows()    
data_rows = [dr for dr in data_rows if dr['DataRow Workflow Info']['taskName'] == "Done"]
print(f'{len(data_rows)} done data rows')

for i, row in enumerate(data_rows):
        
    row_id = row['DataRow ID']
    labels = row['Label']['objects']
    metadata = json.loads(row['Labeled Data'])
    rgb_cog_url = metadata['tileLayerUrl']
    job_name = rgb_cog_url.split('/')[-1].replace('.tif', '')
    
    rgb_path = f'{rgb_dir}/{job_name}.tif'
    composite_path = f'{source_dir}/{job_name}.tif'
    label_path = f'{labels_dir}/{job_name}.tif'

    with rasterio.open(composite_path) as src:
        bbox = list(src.bounds)
        composite_shape = src.shape
        composite_transform = src.transform
    
    label_polygons = {
        label['featureId']: Polygon(label['geometry']['coordinates'][0])
        for label in labels
    }
        
    # 0: no_cloud, 1: cloud
    labels_data = np.zeros(composite_shape).astype(np.uint8)
    
    islands = []    
    sorted_labels = sorted(labels, key=lambda lab: lab['value'], reverse=True)   
    for label in sorted_labels:
        class_value = label['value']
        class_idx = CLOUD_CLASSES[class_value]
                
        label_id = label['featureId']
        polygon = label_polygons[label_id]
        polygon_mask = geometry_mask([polygon], composite_shape, composite_transform, invert=True)
        
        test_polygons = label_polygons.copy()
        del test_polygons[label_id]
        is_poly_island = is_island(polygon, list(test_polygons.values()))
                
        if is_poly_island:
            islands.append((class_idx, polygon_mask))
        else:
            labels_data[polygon_mask] = class_idx

    
    for class_idx, polygon_mask in islands:
        labels_data[polygon_mask] = class_idx
        
    write_array_to_tif(labels_data, label_path, bbox, dtype=np.uint8, nodata=255)

    # plot
    if i == 36:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 12))
        with rasterio.open(rgb_path) as src:
            data = src.read()
            plot_bands(data, ax=ax1, bands=[0, 1, 2], transpose=True)

        ax2.imshow(labels_data, vmin=0, vmax=1)
       
        
print('done')

36 done data rows
done


## Data augmentation - rotate images

In [106]:
%reload_ext autoreload
%autoreload 2

import glob
import matplotlib.pyplot as plt
import numpy as np
import rasterio

from common.constants import NODATA_BYTE, NODATA_FLOAT32
import common.utilities.imagery as imagery
import common.utilities.visualization as visualization


label_paths = glob.glob(f'{labels_dir}/S2*[0-9].tif')


for label_path in label_paths:
    rgb_path = label_path.replace(labels_dir, rgb_dir)
    source_path = label_path.replace(labels_dir, source_dir)
        
    with rasterio.open(source_path) as source_src:
        source_data = source_src.read()
        bbox = list(source_src.bounds)
        
    with rasterio.open(rgb_path) as rgb_src:
        rgb_data = rgb_src.read()
        
    with rasterio.open(label_path) as label_src:
        label_data = label_src.read(1)
        
        
    rgb_target_path = label_path.replace('/labels/', '/rgb/')
    source_target_path = label_path.replace('/labels/', '/source/')

    imagery.write_array_to_tif(rgb_data.transpose((1, 2, 0)), rgb_target_path, bbox, dtype=np.float32, nodata=NODATA_FLOAT32)
    imagery.write_array_to_tif(source_data.transpose((1, 2, 0)), source_target_path, bbox, dtype=np.float32, nodata=NODATA_FLOAT32)
                    
    for i in range(1, 4):
        
        source_rotated = np.rot90(source_data, k=i, axes=(1, 2))
        source_rotated = source_rotated.transpose((1, 2, 0))
        
        rgb_rotated = np.rot90(rgb_data, k=i, axes=(1, 2))
        rgb_rotated = rgb_rotated.transpose((1, 2, 0))
        
        label_rotated = np.rot90(label_data, k=i, axes=(0, 1))
        
        source_rotated_path = source_target_path.replace('.tif', f'_{i*90}deg.tif')
        rgb_rotated_path = rgb_target_path.replace('.tif', f'_{i*90}deg.tif')
        label_rotated_path = label_path.replace('.tif', f'_{i*90}deg.tif')
                
        imagery.write_array_to_tif(source_rotated, source_rotated_path, bbox, dtype=np.float32, nodata=NODATA_FLOAT32)
        imagery.write_array_to_tif(rgb_rotated, rgb_rotated_path, bbox, dtype=np.float32, nodata=NODATA_FLOAT32)
        imagery.write_array_to_tif(label_rotated, label_rotated_path, bbox, dtype=np.uint8, nodata=NODATA_BYTE)
        
        #fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 12))
        #plot_bands(rgb_data, ax=ax1, bands=[0, 1, 2], transpose=True)
        #plot_bands(rgb_rotated, ax=ax2, bands=[0, 1, 2], transpose=False)
        #ax1.imshow(label_data, vmin=0, vmax=1)
        #ax2.imshow(label_rotated, vmin=0, vmax=1)        


## Create train and val folders

In [107]:
import os
import shutil
import splitfolders


output_dir = "./data/trainingCloudData"

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

splitfolders.ratio(training_dir, output=output_dir, seed=314, ratio=(.85, .15), group_prefix=None)

print('done')


Copying files: 0 files [00:00, ? files/s]
Copying files: 2 files [00:00, 17.10 files/s]
Copying files: 4 files [00:00, 13.24 files/s]
Copying files: 6 files [00:00, 12.10 files/s]
Copying files: 8 files [00:00, 11.91 files/s]
Copying files: 10 files [00:00, 11.87 files/s]
Copying files: 12 files [00:00, 11.79 files/s]
Copying files: 14 files [00:01, 11.83 files/s]
Copying files: 16 files [00:01, 11.41 files/s]
Copying files: 18 files [00:01, 11.44 files/s]
Copying files: 20 files [00:01, 11.27 files/s]
Copying files: 22 files [00:01, 11.29 files/s]
Copying files: 24 files [00:02, 11.40 files/s]
Copying files: 26 files [00:02, 11.52 files/s]
Copying files: 28 files [00:02, 11.41 files/s]
Copying files: 30 files [00:02, 11.41 files/s]
Copying files: 32 files [00:02, 11.38 files/s]
Copying files: 34 files [00:02, 11.49 files/s]
Copying files: 36 files [00:03, 11.85 files/s]
Copying files: 38 files [00:03, 11.74 files/s]
Copying files: 40 files [00:03, 11.38 files/s]
Copying files: 42 fil

done
